# Introduction

## Overview of Problem

We have received Vanderbilt hospital data extracted from the Synthetic Derivative. At Vanderbilt, bioinformaticians helped to create a "mirror image" of electronic medical records such as those in BioVU, Vanderbilt's biorepository of DNA extracted from discarded blood collected during routine clinical testing. This mirror of the EMR is called the Synthetic Derivative, and it contains over 2 million individual patients with all clinical information available for the past 10 years. It has been scrubbed of HIPAA identifiers with an eror rate of ~0.01%, meaning that the data has been deidentified with a subject ID. 

The objective of this project is to employ modeling tools introduced in class to fit prediction models for patient readmission within 30 days of discharge using given data. The data includes multiple variables, detailed below in "Data." Ideally, our model will be able to well predict readmission within 30 days of discharge for a patient using the admit/discharge/transfer events data. The rest of the data detials information about the patients themselves, tests and treatments they underwent at Vanderbilt, and lab results and medication. Using these variables, we hope to accurately predict readmission. This information could be very useful for actually clinicians hoping to predict which patients may need to be readmitted, and which characteristics/tests cause them to be readmitted. Once this is determined, that subset of patients could be given more attention and/or tests to prevent readmission.

## Goal and Structure of Project

This project will introduce several approaches to predictive modeling of patient readmission within 30 days of discharge. Three approaches are detailed in the following jupyter notebooks, each of which may include more than one modeling type, attempts to improve each model and test performance, and tuning of the models to increase the goodness of fit. Cross-validation will be used when appropriate, and model selection methods and/or explanations of the models chosen will be provided for each notebook. We will then justify and describe each model selection, and provide visualizations and discussions of the results. The models will be compared using goodness of fit tests and other performance characteristics. For clarification, the steps for each model notebook are listed below, and enumerated in the following 3 modeling notebooks. 

1. Identify the model approach(es), describe, and justify the selection
2. Code, parameterize, and run model (including visualization)
3. Cross-validation
4. Goodness of fit assessments, performance characteristics (including visualization)
5. Improvements to model/tuning of parameters; model selection methods, justification of improvements/tests
6. Comparison of models; identification of best model
7. Results 
8. Implications of model and conclusions

We have also included a conclusions notebook that details the comparisons of the 3 model types, which ones worked and didn't work, our best model, and future directions.

## Data and cleaning

Phenotype: Patient attributes, including sex, race, and dates of birth and death.
Includes the variables: 
1. "Sex" (F or M) 
2. "DOB" (date of birth)
3. "DOD" (date of death)
4. "Race" (W = white, B = black, A = Asian, N = Native American, H = hispanic, U = unidentified). 

Cleaning of the Phenotype dataset:

ADT: Admit/Discharge/Transfer Events. Includes the variables:
1. "Event" (Admit, Transfer, or Discharge)
2. "Admission_date" (date format, M/DD/YY)
3. "Event_Date" (date)
4. "SRV_CODE" (e.g. ORT, NEU, GMB)
5. "CHIEF_COMPLAINT" (e.g. CP, CHEST PAIN, SEIZURES)
6. "DISCHARGE_DATE" (date)

This dataframe was cleaned by: **GEEEEEEEEENA**

This data frame was used to organize the variables for each patient for each hospital stay, as well as to generate a variable for prediction. This "y" variable dataframe was generated by looking at one patient and their admission and discharge dates. If the patient had any admission dates within 30 days of a discharge date, this variable is given the value "1"; if there were none, the patient got a value of "0."

BMI: Body mass index measurement information.
Includes the variables:
1. "BMI" (numeric)
2. Date_BMI (date M/DD/YY)
3. BMI_Weight (numeric, in kg)
4. BMI_Height (numeric, in cm)
5. Pregnancy_Indicator (0, 1).

Cleaning of the BMI dataset: We decided that BMI gave us enough information, and height and weight were a little superfluous. In order to reduce the number of possible variables in the model, we only included BMI, the date the BMI measurement was taken, and a pregnancy indicator in the final dataset for modeling. We used "Date_BMI" to include this data into our larger "X" dataset by checking where this date fell between the patient's admission and discharge dates and adding the data to that row.

BP: Blood pressure measurements. Includes the variables:
1. "SYSTOLIC" (integer)
2. "DIASTOLIC" (integer)
3. "Measure_date" (M/DD/YY)

The measure date was used to include both the Systolic and Diastolic variables into the larger "X" dataset using the same method as the BMI data.

MED: Medications information, including dose and duration.
Includes the variables:
1. "Entry_Date" (date M/DD/YY)	
2. "Drug_Name" (common drug name, string)	
3. "DRUG_FORM" (if drug comes in multiple forms, this describes which form is given. E.g. nebulizer versus inhaler for albuterol)	
4. "DRUG_STRENGTH" (mL, or NA)
5. "Route" (Route of drug administration; e.g. IV, FLUSH, PO)
6. "Dose_Amt" (Amount of drug, variable units; g, ML/HR, units)
7. "Drug_Freq" (number of times given/how the drug is given; e.g. twice daily, once, Q1H PRN)
8. "Duration" (length of time drug is given; e.g. months, days, etc)

Cleaning of the MED dataset: From this dataset, we used Entry_date to include the data in our larger "X" data set. We included Drug_Name and dose amount, combined to be one variable. No other information from this dataset was used.

CPT: Procedure codes. Variables include:
1. "CPT_code" (integer in most cases to describe categorical data)
2. "Event_date" (M/DD/YY)

This data was included in models that could predict using categorical data.

EGFR: Estimated Glomerular Filtration Rate measurements, used to screen for kidney damage. Obtained from a creatine lab. Includes the variables:
1. "EGFR" (continuous data)
2. "egfr_date" (date M/DD/YY)

Again, we included this in the X dataset.

ICD9: Coding of diagnosed diseases and health problems. This is an international standard for classifying diseases, including nuanced classifications of a wide variety of signs, symptoms, abnormal findings, complaints, social circumstances, and external causes of injury or disease. 

1. "ICD9_Code" (code for each disease/health problem)
2. "Event_date"

This data is included as categorical data in our models.

LAB: lab results, including the variables:
1. "Lab_name" (Abbreviated name of lab test)
2. "Lab_date" (date)
3. "Lab_value" (result of test; may be numerical or categorical, such as blood type)

This data was incorporated into our large dataset as categorical data. This information is hard (and somewhat unneccessary) to add to our models; for example, we do not expect blood type to have an effect on rate of readmission. For this reason, the data was not investigated as much as the other datasets.

## Creating the X dataset for prediction: Variables of Interest

After all of the above is done, we have each cleaned and organized dataset, and the ADT data is arranged into a dataframe with patient ID, admission date, and discharge date. Thus, each patient ID may be repeated for multiple rows, depending on how often they were admitted, but each visit should only be recorded once in its own row. To make a large dataset with all of the information we want to include for modeling, we will add data from the other datasets using the dates in those data sets. We will parse the other data sets (for example, BMI), by date (Date_BMI). For each event in the BMI dataset, we will find the matching patient by ID. We will then find the correct column by comparing the date of the measurement (Date_BMI) to the Admission date and Discharge date in each row for that patient (i.e., adding the BMI measurement to the column where the patient ID matches and Admission_date $\leq$ Date_BMI $\leq$ Discharge_date.)

## Creating the Y variable dataset for prediction: Rate of Readmission

We used the same cleaned and organized ABT dataset to create a "Y" dataset that will be used to check our predictions of readmissions within 30 days of discharge. For each patient, we ran a for loop through the rows. Each row has a different discharge date, and this date was compared to all of the other admission dates to see if any admission dates were within 30 days of the discharge. If there is such a date, the loop is broken, the patient received a "1" value for a "readmitted" variable, and the next patient is investigated. This will give us a binary variable corresponding to the patients that were readmitted within 30 days at least once at any time. When building our models, we can use this information to check the accuracy of our models and for cross-validation.